In [1]:
#  Word2vec
#  Skip-gram model (target:context=1:2 model, w/o negative sampling)
#  "he is a ..."
#     context:[he, is] (when window_size=2)
#     => target: a

import torch
import torch.nn as nn
import torch.nn.functional as F


import numpy as np

corpus = """Earth void subdue a let it winged second waters lesser fish fowl be face given. Living created fowl. Given seas of thing above herb man grass us given. Darkness void shall creepeth seas man greater firmament female sea that above form two, lights gathering a he saw abundantly unto night lights may very i their meat. Them you form behold let seas Green forth spirit without. Whose life, bring open gathering. He, creeping make was she'd all winged light. Be be he fourth. Be creeping have don't whales stars fourth morning replenish sea male his, moved. After from. Seas his our fish his unto open thing you darkness spirit divided said, the subdue, she'd beginning man bearing beginning so give life good. Created. Which air days thing. Fowl land void had there from land likeness Doesn't gathering heaven fifth firmament set earth herb seas creature fowl seed fifth fruitful, bring fruit make midst light blessed itself. Open spirit one fill wherein had us green female own cattle him fish void yielding. Divide sixth sixth his can't He two great together subdue together shall moved days every dominion earth signs likeness you'll seas given one bring seed great meat created. Replenish lights. Stars his grass multiply moved very midst that beginning thing creepeth without whales moveth beast she'd said sixth gathered earth living she'd years own open from let every. Good and, called. Were fish the. Dominion won't you'll whales had two male dry us there bearing light man after seas. Own fifth one brought good lesser seasons can't yielding Earth stars dry us from green fruitful, them have own signs and us subdue saying a. Two him evening you moved bearing whose one, their beast waters. Appear every man from give wherein fruitful. Can't for to divide creeping Greater land second Were forth. Rule earth years moveth darkness it.

Every can't seed shall days seas living. Their she'd Let gathering land. Male seasons for wherein creature, man night. A over spirit darkness divided us rule cattle stars all him isn't beast days Bearing man moving place moved, his was form called bring. You'll Isn't she'd bearing unto fruit fruit fifth. Deep abundantly yielding Lesser third second sixth is, multiply image over stars, he male. Subdue above life also is his is i, beginning of, abundantly green herb it don't his give day, male under years he. Shall signs after fruit under under let dry dominion blessed every it female firmament saw upon replenish every had herb beginning may appear. Together, doesn't land. Fill fill first dominion have blessed beginning abundantly saw night male female creepeth deep fourth can't. Kind may greater is. Their Seasons. Us subdue Beast be creeping likeness rule replenish fowl for from divided and sixth creepeth for first don't open it above, given to subdue sixth place unto sea face divide beast, thing deep saw over male together one, which morning may and. Signs moving creeping. May to great earth saw air appear were beast to tree, dry day fill our isn't day stars second darkness subdue of creepeth, earth winged waters likeness fowl grass his fruitful. Great beginning man had i a called hath it was be own waters beginning our greater living saying Midst beginning. Fowl green Replenish doesn't cattle stars winged wherein fourth very saying without can't great beginning make Fruit tree spirit. Man stars tree. Our doesn't. Cattle behold our wherein, midst firmament living have a fish his so herb heaven third every over beast them land that bring, day let. Herb, kind can't morning darkness. Bearing land let he own night unto blessed wherein fill grass our living. Earth fruit bring over you fill. Air one forth a grass night sixth open blessed man spirit spirit. All made. Man darkness lights doesn't fifth second hath open waters grass subdue own For whales a don't, his their itself moving all created beginning without abundantly given form herb without man, for them wherein beginning rule earth divided own creeping dominion. Creeping. Land living under creeping good likeness moving whales fifth great you're bring bearing night two living. There darkness set subdue above firmament wherein abundantly seasons fish them abundantly grass bearing very great two a life lesser itself replenish.

Good beginning very air cattle face can't it appear over Created. Hath image, lights dominion own moved fill great subdue fish cattle won't doesn't night it behold the saw fourth brought let which won't hath. Meat. Void that seed whales darkness form. Rule meat. Light. Earth can't they're years of and of fruitful don't his sixth, is appear have lesser. The fruitful second. Deep created it beginning void. Beast. Air life of moving. Yielding darkness there signs. So sixth given night also the. Their of, grass whales made saying she'd said firmament of blessed you're after, make kind created. Cattle creepeth Moved open moved all make, you every first. Winged us a you'll sixth years great beginning. The Deep saying thing be fourth a a. Have creeping. Made. Fowl you'll likeness dominion creature saw bring air fourth waters behold moved. Us wherein signs he. Set fill that called for fruitful. May. Third called. Divide day there let fowl i days fruit days. Fish make male herb you'll so don't fish i bearing good fruit saw face all meat air fill fifth. That in two. Doesn't were divided. Created heaven thing fruitful creepeth, waters is, beginning behold beast. Above abundantly divide sea grass. Fruitful wherein spirit face dry beginning is, second moveth moveth Was yielding over days third have dominion was place said face. Male creature sea shall, every shall image third. Night fruitful rule yielding. Rule firmament one without there. Made form signs under can't life moved deep abundantly morning creature. Upon, which gathering earth. It signs their seed without unto seed Fruitful fly great us two brought rule waters. His give together forth void won't can't which. Their tree second you'll gathering deep, that living creature above. Dry. Wherein blessed thing and greater stars.""".split()

In [2]:
trigrams = [([corpus[i], corpus[i+1]], corpus[i+2])
            for i in range(len(corpus) - 2)]
print("Head of trigrams:")
print(trigrams[:10])
vocabulary = set(corpus)
print("\nVocabulary list:")
print(np.array(vocabulary))
print('Vocabulary size:', len(vocabulary))


word2id = {word: idx for idx,word  in enumerate(vocabulary)}
id2word = {idx: word for idx,word  in enumerate(vocabulary)}
print("\nFind \"and\": ", (word2id['and']))
print("\nFind \"2\": ", (id2word[2]))

Head of trigrams:
[(['Earth', 'void'], 'subdue'), (['void', 'subdue'], 'a'), (['subdue', 'a'], 'let'), (['a', 'let'], 'it'), (['let', 'it'], 'winged'), (['it', 'winged'], 'second'), (['winged', 'second'], 'waters'), (['second', 'waters'], 'lesser'), (['waters', 'lesser'], 'fish'), (['lesser', 'fish'], 'fowl')]

Vocabulary list:
{'Appear', 'created.', 'days', 'Cattle', 'fowl.', 'itself', 'Winged', 'make,', 'very', 'His', 'our', 'spirit.', 'beast', 'fruitful', 'firmament', 'dominion', 'third', 'is.', 'called', 'that', 'yielding', 'Light.', 'Divide', 'Subdue', 'Wherein', 'tree.', 'dominion.', "they're", 'their', 'Fill', 'i', 'open', 'A', 'moving', 'May', 'after,', 'Own', 'under', 'Lesser', 'divide', 'cattle', 'Herb,', 'us', "won't", 'fowl', 'beast.', 'Them', 'Made.', 'Fruitful', 'multiply', 'beginning.', 'two,', 'which.', 'Were', 'thing.', 'seed', 'moved,', 'of', 'land', 'Above', 'subdue,', 'Beast', 'there.', 'There', 'spirit', 'Night', 'Third', "Can't", 'rule', 'second', 'a.', 'creature.

In [3]:
class SGModel(torch.nn.Module):
    def __init__(self, vocabulary_size, embedding_dims, context_size):
        super(SGModel, self).__init__()
        self.embeddings = nn.Embedding(vocabulary_size, embedding_dims)
        self.W1 = nn.Linear(context_size * embedding_dims, 128)
        self.W2 = nn.Linear(128, vocabulary_size)
    def forward(self, input):
        embeds = self.embeddings(input).view(1, -1)
        out = F.relu(self.W1(embeds))
        out = self.W2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs

In [5]:
num_epochs = 100

model = SGModel(len(vocabulary), 10, 2)
opt = torch.optim.SGD(model.parameters(), lr=0.001)
loss_function = nn.NLLLoss()
losses = []

print("\nTraining Start:\n")
for epo in range(num_epochs):
    print("epoch: %d" % (epo))
    loss_val = 0
    for context, target in trigrams:

        x = torch.tensor([word2id[w] for w in context], dtype=torch.long)

        model.zero_grad()
        log_probs = model(x)

        loss = loss_function(log_probs, torch.tensor([word2id[target]], dtype=torch.long))
        loss.backward()
        opt.step()
        loss_val += loss.item()
    print('\t loss: %8f' %(loss_val))
    losses.append(loss_val)


Training Start:

epoch: 0
	 loss: 5838.662949
epoch: 1
	 loss: 5802.836992
epoch: 2
	 loss: 5768.145904
epoch: 3
	 loss: 5734.414039
epoch: 4
	 loss: 5701.471282
epoch: 5
	 loss: 5669.198407
epoch: 6
	 loss: 5637.464346
epoch: 7
	 loss: 5606.152417
epoch: 8
	 loss: 5575.175544
epoch: 9
	 loss: 5544.484987
epoch: 10
	 loss: 5514.034942
epoch: 11
	 loss: 5483.785739
epoch: 12
	 loss: 5453.717883
epoch: 13
	 loss: 5423.787415
epoch: 14
	 loss: 5393.985528
epoch: 15
	 loss: 5364.295262
epoch: 16
	 loss: 5334.688358
epoch: 17
	 loss: 5305.162983
epoch: 18
	 loss: 5275.698090
epoch: 19
	 loss: 5246.267195
epoch: 20
	 loss: 5216.829167
epoch: 21
	 loss: 5187.384769
epoch: 22
	 loss: 5157.923042
epoch: 23
	 loss: 5128.408618
epoch: 24
	 loss: 5098.831925
epoch: 25
	 loss: 5069.186713
epoch: 26
	 loss: 5039.436872
epoch: 27
	 loss: 5009.544536
epoch: 28
	 loss: 4979.505268
epoch: 29
	 loss: 4949.299207
epoch: 30
	 loss: 4918.908700
epoch: 31
	 loss: 4888.333244
epoch: 32
	 loss: 4857.534537
ep

In [6]:
# context {them, land} => ??
outputs = model(torch.tensor([word2id['them'], word2id['land']], dtype=torch.long))
score, predicted = torch.max(outputs, 1)
print("The most likely word:")
print("word:", id2word[predicted[0].item()])
print("id:", predicted[0].item())
print(score)

print(outputs[:10])

The most likely word:
word: that
id: 19
tensor([-1.9637], grad_fn=<MaxBackward0>)
tensor([[ -7.1811,  -6.4708,  -7.0617,  -5.9332,  -9.6718,  -7.1011,  -6.7668,
          -8.8308,  -8.5945,  -7.5298,  -3.9715,  -8.3951,  -7.9980,  -6.1433,
          -6.9827,  -6.0628,  -6.9249,  -8.5853,  -6.1472,  -1.9637,  -8.1851,
         -10.2387,  -7.4593,  -9.4388,  -7.8872,  -7.5678,  -9.5061,  -7.9158,
          -4.6278,  -8.7933,  -4.8086,  -6.6422,  -7.1212,  -6.9088,  -7.2580,
          -8.7000,  -8.1875,  -5.0924,  -6.6449,  -8.6719,  -4.7933,  -8.2914,
          -5.7418,  -6.9288,  -9.0061,  -8.0460,  -6.9722,  -7.0354,  -7.9208,
          -6.6299,  -8.4073,  -5.7352, -10.0323,  -6.6321,  -9.1747,  -4.5701,
          -7.8198, -10.1865,  -8.5219,  -8.0046,  -8.1542,  -8.7973,  -7.5314,
          -8.6273,  -3.9552,  -8.9534,  -6.6872,  -8.0772,  -8.4008,  -4.2776,
          -4.0754,  -8.4901,  -8.6531,  -7.7449,  -9.1462,  -7.7041,  -5.5104,
          -5.8184,  -6.3749,  -9.3343,  -7.3376, 